[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/NMF.ipynb)

# 非負値行列分解

In [1]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [2]:
# 欠損値の穴埋め方法
fillna_with_zero = True
# 因子数
factors = 5

In [4]:
# 評価値をユーザー×映画の行列に変換。欠損値は、平均値または０で穴埋めする
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
if fillna_with_zero:
    matrix = user_movie_matrix.fillna(0).to_numpy()
else:
    matrix = user_movie_matrix.fillna(movielens_train.rating.mean()).to_numpy()
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [6]:
from sklearn.decomposition import NMF

# NMFの実行
nmf = NMF(n_components=factors)
nmf.fit(matrix)


/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)
/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


NMF(n_components=5)

In [7]:
P = nmf.fit_transform(matrix)
P

/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)
/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


array([[0.02183305, 0.53589333, 0.        , 0.        , 0.        ],
       [0.21316287, 0.10881497, 0.        , 0.        , 0.        ],
       [0.        , 0.02976101, 0.05290754, 0.15364687, 0.02014396],
       ...,
       [0.13635948, 0.14281692, 0.        , 0.        , 0.00656248],
       [0.09754   , 0.01077277, 0.        , 0.13238006, 0.13515664],
       [0.99759019, 0.        , 0.        , 0.64704357, 0.05969731]])

In [8]:
Q = nmf.components_
Q

array([[1.96418693, 0.50120485, 0.39873535, ..., 0.        , 0.        ,
        0.        ],
       [1.15067665, 0.96078243, 0.40197059, ..., 0.        , 0.00289228,
        0.00216921],
       [0.47179502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.79016702, 0.3041748 , 0.        , ..., 0.02803924, 0.01446954,
        0.01085215],
       [0.29612837, 0.30057646, 0.17845097, ..., 0.        , 0.        ,
        0.        ]])

In [10]:
import numpy as np
# 予測評価値行列
pred_matrix = np.dot(P, Q)
pred_matrix

array([[6.59524137e-01, 5.25819727e-01, 2.24118966e-01, ...,
        0.00000000e+00, 1.54995325e-03, 1.16246494e-03],
       [5.43902575e-01, 2.11385779e-01, 1.28735991e-01, ...,
        0.00000000e+00, 3.14723309e-04, 2.36042481e-04],
       [1.86578695e-01, 8.13841578e-02, 1.55577588e-02, ...,
        4.30814090e-03, 2.30927602e-03, 1.73195702e-03],
       ...,
       [4.34114948e-01, 2.07532549e-01, 1.12950629e-01, ...,
        0.00000000e+00, 4.13066441e-04, 3.09799830e-04],
       [3.48608841e-01, 1.40129388e-01, 6.73418166e-02, ...,
        3.71183579e-03, 1.94663585e-03, 1.45997689e-03],
       [2.48840417e+00, 7.14754992e-01, 4.08427521e-01, ...,
        1.81426079e-02, 9.36242017e-03, 7.02181513e-03]])

In [13]:
# 学習用に出てこないユーザーや映画の予測評価値は、平均評価値とする
average_score = movielens_train.rating.mean()
movie_rating_predict = movielens_test.copy()
pred_results = []
for i, row in movielens_test.iterrows():
    user_id = row["user_id"]
    if user_id not in user_id2index or row["movie_id"] not in movie_id2index:
        pred_results.append(average_score)
        continue
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    pred_score = pred_matrix[user_index, movie_index]
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

In [14]:
from collections import defaultdict

# 各ユーザに対するおすすめ映画は、そのユーザがまだ評価していない映画の中から予測値が高い順にする
pred_user2items = defaultdict(list)
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in movielens_train.user_id.unique():
    if user_id not in user_id2index:
        continue
    user_index = user_id2index[row["user_id"]]
    movie_indexes = np.argsort(-pred_matrix[user_index, :])
    for movie_index in movie_indexes:
        movie_id = user_movie_matrix.columns[movie_index]
        if movie_id not in user_evaluated_movies[user_id]:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {139: [4993, 5952, 4226, 7153, 4306, 3996, 6539, 5445, 6874, 4963],
             149: [4226, 2571, 2762, 47, 3996, 318, 6539, 5445, 4963, 1704],
             182: [47, 6874, 2329, 3793, 4011, 7361, 4878, 7438, 6333, 2502],
             215: [2329, 527, 6711, 3147, 2683, 8360, 5669, 6502, 3948, 1246],
             281: [2959, 4993, 5952, 4226, 2571, 2858, 7153, 3578, 2762, 4306],
             326: [2959, 4993, 5952, 4226, 2571, 2858, 7153, 3578, 2762, 47],
             351: [2959, 4993, 5952, 4226, 7153, 3578, 2762, 47, 4306, 3996],
             357: [2959, 4993, 5952, 4226, 2858, 7153, 3578, 2762, 296, 47],
             426: [2502,
              33794,
              3949,
              7147,
              33493,
              5902,
              44191,
              33166,
              4848,
              6016],
             456: [2959, 4993, 5952, 4226, 2571, 2858, 7153, 3578, 2762, 4306],
             459: [318, 4886, 2997, 50, 4995, 4973, 7361, 4878, 3

In [15]:
# user_id=2のユーザーが学習データで評価を付けた映画一覧
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [16]:
pred_user2items[2]

[2959, 4993, 5952, 4226, 2571, 2858, 7153, 3578, 2762, 296]

In [17]:
# user_id=2に対するおすすめ(2959, 4993, 5952)
movies[movies.movie_id.isin([2959, 4993, 5952])]

,movie_id,title,genre,tag
2874,2959,Fight Club (1999),"[Action, Crime, Drama, Thriller]","[based on a book, chuck palahniuk, edward nort..."
4899,4993,"Lord of the Rings: The Fellowship of the Ring,...","[Action, Adventure, Fantasy]","[based on a book, big budget, new zealand, sce..."
5852,5952,"Lord of the Rings: The Two Towers, The (2002)","[Action, Adventure, Fantasy]","[based on a book, big budget, new zealand, sce..."
